In [1]:
import cv2
import numpy as np
import imutils
import math
from itertools import combinations

# Euclidean distance calculation fuction with points (x_1,y_1) and (x_2,y_2)
def distance(x_1,y_1,x_2,y_2):
    dist = ((x_1-x_2)**2 + (y_1-y_2)**2)
    return math.sqrt(dist)

# Function to identity interested points (used to get aspect ratio and corner points)
def mouse_points(event,x,y,flags,params):
    if event == cv2.EVENT_LBUTTONDBLCLK:
        print(x,y)

wht = 320   # width, height , target - all set to 320

path = 'pedestrian_walking01.mkv'  # path to video feed
# path = 'pedestrian_walking02.mkv'
# path = 'pedestrian_walking03.mkv'

# initializing webcam/video feed
cap = cv2.VideoCapture(path)

# pre-trained configuration and weights
model_config = 'yolov3_320.cfg'       # yolov3_tiny.cfg also used for faster fps but low accuracy
model_weights = 'yolov3_320.weights'  # yolov3-tiny.weights also used for faster fps and low accuracy

# model_config = 'yolo-obj.cfg'
# model_weights = 'yolo-obj_5000.weights'

# Network
net = cv2.dnn.readNetFromDarknet(model_config,model_weights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Thresholds
conf_threshold = 0.3   # confidence threshold
nms_threshold = 0.1    # Non-maxima supression threshold
dist_threshold = 150   # Safe distance threshold (applied randomly)

# for polylines on frame of video feed (yellow lines)
pts = np.asarray([[475,15],[823,29],[738,447],[47,269]],np.int32)
pts = pts.reshape((-1, 1, 2)) 

# Transformation matrix
pts_1 = np.float32([[475,15],[823,29],[738,447],[47,269]]) # points are taken from the video by using mouse_points function
pts_2 = np.float32([[0,0],[700,0],[700,800],[0,800]])      # aspect ratio: calculated using points taken from mouse_points function
matrix = cv2.getPerspectiveTransform(pts_1,pts_2)

def find_objects(outputs,img,frame,matrix):
    ht,wt,ct = frame.shape # target shapes
    bbox = []              # Bounding boxes
    class_Ids = []         # class_Ids (tags)
    confs = []             # confidence in object detection

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_Id = np.argmax(scores)
            confidence = scores[class_Id]
            if confidence > conf_threshold and class_Id ==0:
                w, h = int(detection[2]*wt), int(detection[3]*ht)                  # multiplying with wt and ht as detection is in percentages
                x, y = int((detection[0]*wt) - w/2),int((detection[1]*ht)-h/2)
                cx, cy = int(detection[0]*wt), int(detection[1]*ht)                # centre X and ground Y
                arr = np.asarray([[cx],
                                  [int(cy+h/2)],
                                  [1]])
                warp_arr = matrix@arr                                              # warp_arr contains the tranformation of the centre X and ground Y points on to                                                                                       # the warped plane.

                bbox.append([x,y,w,h,cx,cy,int(warp_arr[0][0]*0.479),int(warp_arr[1][0]*0.453)]) # scaling ratio - ratio of width and height in original feed and 
                                                                                                 # the warped width and height.
                class_Ids.append(class_Id)
                confs.append(float(confidence))
    
    indices = cv2.dnn.NMSBoxes(bbox,confs,conf_threshold,nms_threshold)
    
    dist_dict = {}        # contains person_ID w.r.t frame and their warp tranform coordinates
    for i in indices:
        i = i[0]
        box = bbox[i]
        x,y,w,h,cx,cy,wcx,wcy = box[0],box[1],box[2],box[3],box[4],box[5],box[6],box[7]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,255),2)
        cv2.circle(frame,(int(cx),int(cy+(h/2))),2,(0,0,255),-1)
        cv2.circle(img,(int(wcx),int(wcy)),3,(0,0,255),-1)
        
        dist_dict[i] = (box[6],box[7])
        
    red_zone = []       # All IDs not maintaining social distance threshold w.r.t current frame.
    green_zone = []     # All IDs maintaining social distance threshold w.rt. current frame.
    for (id_1,p_1),(id_2,p_2) in combinations(dist_dict.items(),2):
        dx, dy = p_1[0]-p_2[0], p_1[1]-p_2[1]
        distances = math.sqrt(dx*dx + dy*dy)
        if distances < dist_threshold:
            if id_1 not in red_zone:
                red_zone.append(id_1)
            if id_2 not in red_zone:
                red_zone.append(id_2)
        else:
            if id_1 not in red_zone:
                green_zone.append(id_1)
            if id_2 not in red_zone:
                green_zone.append(id_2)
        
    if len(red_zone)>=2:
        for i,j in combinations(red_zone,2):
            cv2.line(img,dist_dict[i],dist_dict[j],(0,0,255),2)
    if len(green_zone)>=2:
        for i,j in combinations(green_zone,2):
            cv2.line(img,dist_dict[i],dist_dict[j],(0,255,0),1)

    
while True:
    _,frame = cap.read()
    
    blob = cv2.dnn.blobFromImage(frame,1/255,(wht,wht),[0,0,0],1,crop = False)
    net.setInput(blob)
    
    layer_names = net.getLayerNames()
    output_names = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
    
    cv2.polylines(frame,[pts],isClosed = True,color = (0,255,255),thickness = 1)
    outputs = net.forward(output_names)

    img = cv2.warpPerspective(frame,matrix,(700,800))
    img[0:800,0:700] = [0,0,0]

    find_objects(outputs,img,frame,matrix)

    img_ = imutils.resize(img,480)
    cv2.imshow('sample',frame)
    cv2.imshow('warped',img_)
    key = cv2.waitKey(1)
    
    if key == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()